In [1]:
from transformers import PreTrainedModel, AutoModelForSeq2SeqLM

/data/soyeonhong/anaconda3/envs/groundvqa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lm = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [6]:
from transformers.models.t5.modeling_t5 import T5Stack